# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.

In [7]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00


In [8]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [9]:
#import os
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
#HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY_Ironhack')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_TOKEN')

In [11]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [33]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "/content/sonnets.txt")

In [34]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [35]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [20]:
from langchain_community.document_loaders import WebBaseLoader

In [36]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [37]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [23]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [24]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [25]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=3, verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [26]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer.
Final Answer: Biden nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "Biden nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [ ]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.
Thought: I now know the final answer
Final Answer: Ruff offers more features and flexibility compared to flake8, making it a better choice for linting and formatting Python code.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers more features and flexibility compared to flake8, making it a better choice for linting and formatting Python code.'}

In [27]:
# Results with updated code
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it's worth exploring the differences.
Action: Ruff QA System
Action Input: "What are the unique features of ruff compared to flake8?"
Observation:  Ruff has over 800 implemented rules, compared to Flake8's ~409 rules. Ruff also has a different rule code and prefix system to minimize conflicts and allow for easier toggling of individual plugins. Additionally, Ruff does not support custom lint rules, but instead re-implements popular Flake8 plugins in Rust. Ruff also has some minor incompatibilities with Flake8 plugins, but many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, unlike Flake8.
Thought: These unique features of Ruff make it a strong contender for a python linter.
Action: Ruff QA System
Action Input: "What are the benefits of using Ruff over Flake8?"
Observation:  Ruff offers a large

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, compatibility with Black, automatic fixing of lint violations, and support for Python 3.7 onwards, making it a strong contender for a python linter.'}

**Shakespeare's Sonnets**

In [43]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "/content/sonnets.txt")

In [44]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [45]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [46]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

In [47]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare Sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [48]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=3, verbose=True
)

In [49]:
agent.invoke(
    "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?"
)



> Entering new AgentExecutor chain...
 It would be helpful to use a tool that is specifically designed for answering questions about Shakespeare's sonnets.
Action: Shakespeare Sonnets QA System
Action Input: "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?"
Observation:  Some prevalent themes in Shakespeare's sonnets include love, beauty, mortality, and the power of language. These themes reflect the concerns of the Elizabethan era, which was characterized by a fascination with love and beauty, as well as a preoccupation with the fleeting nature of life and the desire for immortality. The use of language and poetry to preserve and immortalize love and beauty was also a common theme in Elizabethan literature. Additionally, the fear of mortality and the inevitability of time's passing were prevalent concerns in this era, as the Elizabethans were living in a time of great political and social change.
Thought: This informat

{'input': "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [50]:
agent.invoke("How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?")



> Entering new AgentExecutor chain...
 I should use the Shakespeare Sonnets QA System to answer this question.
Action: Shakespeare Sonnets QA System
Action Input: "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?"
Observation:  Shakespeare uses the imagery of a summer's day to compare the beauty of the subject to the fleeting nature of the season. He also uses the metaphor of eternal summer to represent the subject's everlasting beauty and the idea that their beauty will never fade.
Thought: I now know the final answer.
Final Answer: Shakespeare uses imagery and metaphor in Sonnet 18 to convey the message that the subject's beauty is eternal and will never fade. 

> Finished chain.


{'input': "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?",
 'output': "Shakespeare uses imagery and metaphor in Sonnet 18 to convey the message that the subject's beauty is eternal and will never fade."}

In [51]:
agent.invoke("What role does the concept of time play in Shakespeare's sonnets, and how does the poet address its impact on beauty and love?"
)





> Entering new AgentExecutor chain...
 It would be helpful to use the Shakespeare Sonnets QA System to answer this question.
Action: Shakespeare Sonnets QA System
Action Input: "What role does time play in Shakespeare's sonnets? How does the poet address its impact on beauty and love?"
Observation:  Time is a major theme in Shakespeare's sonnets, as the poet reflects on the fleeting nature of beauty and the inevitability of aging and decay. He compares human life to plants, which grow and eventually wither away, and warns his beloved to beware of the power of time to take away their youth and beauty. However, he also offers a solution to this problem by suggesting that through love, one can transcend the limitations of time and find eternal beauty.
Thought: Now I have a better understanding of the role of time in Shakespeare's sonnets.
Final Answer: The concept of time plays a significant role in Shakespeare's sonnets, as the poet explores its impact on beauty and love and offers a s

{'input': "What role does the concept of time play in Shakespeare's sonnets, and how does the poet address its impact on beauty and love?",
 'output': "The concept of time plays a significant role in Shakespeare's sonnets, as the poet explores its impact on beauty and love and offers a solution through the power of love."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [41]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about Shakespeare. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [42]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [30]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [31]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it may be useful in certain situations.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.'}

**Shakespeare Sonnets**

In [59]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [60]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [61]:
agent.invoke("What role does the concept of time play in Shakespeare's sonnets, and how does the poet address its impact on beauty and love?"
)




> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What role does the concept of time play in Shakespeare's sonnets, and how does the poet address its impact on beauty and love?"
Observation:  The concept of time is a recurring theme in Shakespeare's sonnets, as the poet reflects on the fleeting nature of beauty and the inevitability of aging and death. In Sonnet XVI, the poet compares human life to plants that grow and eventually wither away, and in Sonnet XIII, he laments the passing of time and its effect on beauty. However, the poet also offers a solution to this struggle against time, suggesting that through procreation, one can leave a lasting legacy and defy the inevitable decay of beauty.


> Finished chain.


{'input': "What role does the concept of time play in Shakespeare's sonnets, and how does the poet address its impact on beauty and love?",
 'output': " The concept of time is a recurring theme in Shakespeare's sonnets, as the poet reflects on the fleeting nature of beauty and the inevitability of aging and death. In Sonnet XVI, the poet compares human life to plants that grow and eventually wither away, and in Sonnet XIII, he laments the passing of time and its effect on beauty. However, the poet also offers a solution to this struggle against time, suggesting that through procreation, one can leave a lasting legacy and defy the inevitable decay of beauty."}

In [62]:
agent.invoke(
    "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?"
)




> Entering new AgentExecutor chain...
 It would be helpful to use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?"
Observation:  Some prevalent themes in Shakespeare's sonnets include love, beauty, mortality, and the power of language. These themes reflect the concerns of the Elizabethan era, which was characterized by a fascination with love and beauty, as well as a preoccupation with the fleeting nature of life and the desire for immortality. The use of language and poetry as a means of preserving love and beauty also reflects the importance placed on literature and the arts during this time period. Additionally, the sonnets often explore the complexities of relationships and the societal expectations placed on individuals, reflecting the strict social norms and expectations of the Elizabethan era.


> Finished chain.


{'input': "What themes are prevalent in Shakespeare's sonnets, and how do they reflect the concerns of the Elizabethan era?",
 'output': " Some prevalent themes in Shakespeare's sonnets include love, beauty, mortality, and the power of language. These themes reflect the concerns of the Elizabethan era, which was characterized by a fascination with love and beauty, as well as a preoccupation with the fleeting nature of life and the desire for immortality. The use of language and poetry as a means of preserving love and beauty also reflects the importance placed on literature and the arts during this time period. Additionally, the sonnets often explore the complexities of relationships and the societal expectations placed on individuals, reflecting the strict social norms and expectations of the Elizabethan era."}

In [58]:
agent.invoke("How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?"
Observation:  Shakespeare uses the imagery of a summer's day to compare the beauty of the subject to the fleeting nature of the season. He also uses the metaphor of eternal summer to represent the subject's everlasting beauty and the idea that their beauty will never fade.


> Finished chain.


{'input': "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?",
 'output': " Shakespeare uses the imagery of a summer's day to compare the beauty of the subject to the fleeting nature of the season. He also uses the metaphor of eternal summer to represent the subject's everlasting beauty and the idea that their beauty will never fade."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [32]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the State of Union QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I should use the State of Union QA System to answer the second question.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: I now know the final answer.
Final Answer: No, the president did not mention a specific tool in the state of the union.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, the president did not mention a specific tool in the state of the union.'}

In [63]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [64]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [65]:
agent.invoke("How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?"
Observation:  Shakespeare uses imagery and metaphor to compare the fleeting nature of a summer's day to the eternal beauty of the subject of the poem. He also uses the image of winter to represent the eventual decline of beauty, but suggests that the subject's beauty will live on through his words.
Thought: I now know the final answer.
Final Answer: Shakespeare uses imagery and metaphor to compare the fleeting nature of a summer's day to the eternal beauty of the subject of the poem. He also uses the image of winter to represent the eventual decline of beauty, but suggests that the subject's beauty will live on through his words.

> Finished chain.


{'input': "How does Shakespeare use imagery and metaphor in Sonnet 18 (Shall I compare thee to a summers day?) to convey the poem's message?",
 'output': "Shakespeare uses imagery and metaphor to compare the fleeting nature of a summer's day to the eternal beauty of the subject of the poem. He also uses the image of winter to represent the eventual decline of beauty, but suggests that the subject's beauty will live on through his words."}

In [66]:
agent.invoke("What's ruff and where to use it?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that helps identify and fix errors in code. It can be used in any python project.
Action: Ruff QA System
Action Input: "What is ruff and where can it be used?"
Observation:  Ruff is a tool that can be used for linting and formatting Python code. It can be installed and used on any Python version from 3.7 onwards.
Thought: This is helpful information, but it doesn't fully answer the question.
Action: Ruff QA System
Action Input: "What is the purpose of ruff?"
Observation:  The purpose of Ruff is to provide a linter and formatter for Python code that is compatible with Flake8 plugins, but also has its own unique rule set and limitations. It also aims to minimize conflicts between plugins and allow for easy toggling of individual plugins.
Thought: This provides more insight into the specific purpose of ruff, but still doesn't fully answer the question.
Action: Ruff QA System
Action Input: "Where can ruff be used?"
Observatio

{'input': "What's ruff and where to use it?",
 'output': 'Ruff is a python linter that can be used in any python project. It is available on PyPI and aims to provide a linter and formatter for Python code with its own unique rule set and compatibility with Flake8 plugins.'}